## Base Model

Model: https://huggingface.co/matsuo-lab/weblab-10b

In [ ]:
!pip install transformers accelerate sentencepiece bitsandbytes --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.8 MB/s eta 0:00:00


In [ ]:
# Wrap printed text
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

## Playing with the Model

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained("matsuo-lab/weblab-10b")
model = AutoModelForCausalLM.from_pretrained(
    "matsuo-lab/weblab-10b",
    device_map="auto",
    quantization_config=quantization_config,
)

# if torch.cuda.is_available():
#     model = model.to("cuda")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
model

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 4864)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-35): 36 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((4864,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((4864,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear4bit(in_features=4864, out_features=14592, bias=True)
          (dense): Linear4bit(in_features=4864, out_features=4864, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear4bit(in_features=4864, out_features=19456, bias=True)
          (dense_4h_to_h): Linear4bit(in_features=19456, out_features=

In [ ]:
model.device

device(type='cuda', index=0)

In [ ]:
tokenizer.pad_token_id = tokenizer.eos_token_id = 0

In [ ]:
text = "坂本龍馬は、"
inputs = tokenizer(text, return_tensors="pt")
inputs

{'input_ids': tensor([[31902,   213, 18957,   165,   124,   224, 40843,   107,  6418,  4746]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
# これを指定しないと走らなかった
tokenizer.encode('<|endoftext|>', return_tensors='pt')

tensor([[0]])

In [ ]:
tokenizer.vocab_size

50254

In [ ]:
with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=100,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)


坂本龍馬は、幕末の日本を動かしたただ一人のヒーローと言っても過言ではない。
そんな龍馬が暗殺された理由は何であったのか？ その事件に迫る。
【出演者】　司会・田原俊彦、小倉久寛
ＴＢ�


In [ ]:
text = """
例題: りんごが5つあります。そこから2つのりんごを取り除きました。残りのりんごの数は何個でしょう？
回答:
""".strip()
inputs = tokenizer(text, return_tensors='pt')

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=100,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)


例題: りんごが5つあります。そこから2つのりんごを取り除きました。残りのりんごの数は何個でしょう？
回答: 5-2=3
このように、「あるものの中から別のものを取り出す」ということを考える問�りでは、「あるもの」と「別のもの」の関係性を図や表などを用いて視覚的にわかりやすく示した上で、その関係性をもとにして、「あるもの」と「別のもの


In [ ]:
text = """
例題: バットとボールの両方を買うと1100円です。バットはボールよりも1000円高いです。ボールはいくらでしょう？
回答:
""".strip()

inputs = tokenizer(text, return_tensors='pt')

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=100,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)


例題: バットとボールの両方を買うと1100円です。バットはボールよりも1000円高いです。ボールはいくらでしょう？
回答: では、こちらの問題にお進みください。
以上が一般的な授業の流れです。
しかし、実際にはこのような教育は行われていないと思っています。
まず第一に、学級崩壊状態になっていること、すべての生徒や先生の意欲を


In [ ]:
text = """
例題: 引数kを取り、返り値としてフィボナッチ数列におけるk個目の値を返すPython関数を書いてください。
回答:
""".strip()

inputs = tokenizer(text, return_tensors='pt')

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=100,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0], )
print(output)


例題: 引数kを取り、返り値としてフィボナッチ数列におけるk個目の値を返すPython関数を書いてください。
回答: def fibonnaci(k):　for i in range(2, k+1) for j in range(i-1, -1) yield fibonnaci(j)+fibonnaci(i-1)　return fibonnaci(k-1)　print(fibonnaci(10))　# 出力結果: 0 1 1 2 3 5 8 13 21 34 55 実行結果はこちら


In [ ]:
text = """
English: There were 3 apples and 2 oranges. How many fruits were there in total?
日本語:
""".strip()

inputs = tokenizer(text, return_tensors='pt')

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=100,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)


English: There were 3 apples and 2 oranges. How many fruits were there in total?
日本語: リンゴが3つと、オレンジが2つあります。全体で何種顳度のフルーツがあるか答えなさい。
A: There are 5 kinds of fruits.
B: There are 4 kinds of fruits.
C: There are 3 kinds of fruits.
D: There are 2 kinds of fruits.
E: There is 1 kind of fruit.
F: There are 6


In [ ]:
# del model
# torch.cuda.empty_cache()

## Instruction SFT
Model: https://huggingface.co/matsuo-lab/weblab-10b-instruction-sft


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("matsuo-lab/weblab-10b-instruction-sft")
model = AutoModelForCausalLM.from_pretrained(
    "matsuo-lab/weblab-10b-instruction-sft",
    torch_dtype=torch.float16,
    device_map="auto",
)

if torch.cuda.is_available():
    model = model.to("cuda")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
text = "大規模言語モデルについて説明してください。"
text = f'以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n{text}\n\n### 応答:'
token_ids = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=100,
        do_sample=True,
        temperature=0.5,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)


ValueError: ignored

In [ ]:
tokenizer.pad_token_id = tokenizer.eos_token_id = 0

In [ ]:
text = "大規模言語モデルについて説明してください。"
text = f'以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n{text}\n\n### 応答:'
token_ids = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=100,
        do_sample=True,
        temperature=0.5,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)

以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。

### 指示:
大規模言語モデルについて説明してください。

### 応答:大規模言語モデルは、複数の言語を処理するために使用できるコンピュータ・プログラムであり、複数の言語を比較し、その違いを特定し、その言語間の関係を調査するために使用できます。大規模言語モデルは


In [ ]:
text = "k個目のフィボナッチ数列を返関数Pythonプログラムを書きなさい。"
text = f'あなたはPythonのエキスパートです。ユーザーの指示にできるだけ正確に丁寧に答えてください。\n\n### 指示:\n{text}\n\n### 応答:'
token_ids = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=1000,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)

あなたはPythonのエキスパートです。ユーザーの指示にできるだけ正確に丁寧に答えてください。

### 指示:
k個目のフィボナッチ数列を返関数Pythonプログラムを書きなさい。

### 応答:def fibonacci(n):
 if n==0:
 return 0
 elif n==1:
 return 1
 else:
 return fibonacci(n-1)+fibonacci(n-2)
<|endoftext|>
